In [ ]:
# 네이버 블로그 크롤링 - 제주맛집

In [ ]:
from selenium import webdriver
import time
import pandas as pd
from urllib.parse import quote

In [5]:
url = f'https://search.naver.com/search.naver?where=view&sm=tab_jum&query={quote("제주맛집")}'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)
time.sleep(2)

In [6]:
# Blog 페이지를 아래로 내려서 많은 데이터 확보
from selenium.webdriver.common.keys import Keys

body = driver.find_element_by_tag_name('body')
for _ in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [ ]:
li = lis[0]
title = li.find_element_by_css_selector('a.api_txt_lines.total_tit._cross_trigger').text.strip()
title

In [ ]:
date = li.find_element_by_css_selector('.sub_time.sub_txt').text.strip()
date

110

In [ ]:
desc = li.find_element_by_css_selector('.api_txt_lines.dsc_txt').text.strip()
desc

AttributeError: 'NoneType' object has no attribute 'get_text'

In [ ]:
href = li.find_element_by_css_selector('a.api_txt_lines.total_tit._cross_trigger').get_attribute('href')
href

In [ ]:
# 날짜 처리
from datetime import datetime, timedelta

def convert_date(s):
    if len(s) == 11:
        return s[:-1]

    today = datetime.now()
    if '어제' in s:
        date = today - timedelta(1)
    elif '일' in s:
        date = today - timedelta(int(s[:-3]))
    elif '시간' in s:
        date = today - timedelta(hours=int(s[:-4]))
    elif '분' in s:
        date = today - timedelta(minutes=int(s[:-3]))

    return f'{date.year}.{date.month:02d}.{date.day:02d}'

In [ ]:
#반복문으로 모든 데이터 가져오기
lines = []
for li in lis:
    title = li.find_element_by_css_selector('a.api_txt_lines.total_tit._cross_trigger').text.strip()
    date = li.find_element_by_css_selector('.sub_time.sub_txt').text.strip()
    desc = li.find_element_by_css_selector('.api_txt_lines.dsc_txt').text.strip()
    href = li.find_element_by_css_selector('a.api_txt_lines.total_tit._cross_trigger').get_attribute('href')
    lines.append([title,convert_date(date),desc,href])

df = pd.DataFrame(lines, columns=['제목','날짜','내용','href'])
df[['제목','날짜','내용']].head()


In [ ]:
df.to_csv('네이버-제주맛집.csv', index=False)

datetime.datetime(2021, 12, 8, 10, 44, 36, 542573)

In [ ]:
driver.close()

(2021, 12, 8)